In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
import warnings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import f1_score, confusion_matrix
import math

# 경고 메시지 무시
warnings.filterwarnings('ignore')

# 한글 폰트 설정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
fontprop = font_manager.FontProperties(fname=font_path)
rc('font', family=fontprop.get_name())

# 1. 데이터 로드
train = pd.read_csv('/apps/study_promptengineerings/dacon/real_estate_fraud/train.csv')
test = pd.read_csv('/apps/study_promptengineerings/dacon/real_estate_fraud/test.csv')

# ID 매핑 저장 및 컬럼 제거
id_mapping = pd.Series(test['ID'].values, index=test.index)
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

# 결측치 확인
print("Train set missing values:")
print(train.isnull().sum())
print("\nTest set missing values:")
print(test.isnull().sum())


Train set missing values:
매물확인방식      0
보증금         0
월세          0
전용면적      787
해당층       229
총층         16
방향          0
방수         16
욕실수        18
주차가능여부      0
총주차대수     696
관리비         0
중개사무소       0
제공플랫폼       0
게재일         0
허위매물여부      0
dtype: int64

Test set missing values:
매물확인방식      0
보증금         0
월세          0
전용면적      184
해당층        50
총층          2
방향          0
방수          2
욕실수         2
주차가능여부      0
총주차대수     175
관리비         0
중개사무소       0
제공플랫폼       0
게재일         0
dtype: int64


In [3]:

# 2. 결측치 처리
# 2.1 총층 처리
train['총층'] = train['총층'].fillna(train['총층'].median())
test['총층'] = test['총층'].fillna(test['총층'].median())

# 2.2 욕실수 처리
train['욕실수'] = train['욕실수'].fillna(train.groupby('방수')['욕실수'].transform('median'))
test['욕실수'] = test['욕실수'].fillna(test.groupby('방수')['욕실수'].transform('median'))
train['욕실수'] = train['욕실수'].fillna(train['욕실수'].median())
test['욕실수'] = test['욕실수'].fillna(test['욕실수'].median())

# 2.3 해당층 처리
train['해당층'] = train['해당층'].fillna(train['해당층'].median())
test['해당층'] = test['해당층'].fillna(test['해당층'].median())

# 2.4 전용면적 처리
train['전용면적'] = train['전용면적'].fillna(train.groupby(['방수', '욕실수'])['전용면적'].transform('median'))
test['전용면적'] = test['전용면적'].fillna(test.groupby(['방수', '욕실수'])['전용면적'].transform('median'))
train['전용면적'] = train['전용면적'].fillna(train['전용면적'].median())
test['전용면적'] = test['전용면적'].fillna(test['전용면적'].median())

# 2.5 총주차대수 처리
train.loc[train['주차가능여부'] == '불가능', '총주차대수'] = train.loc[train['주차가능여부'] == '불가능', '총주차대수'].fillna(0)
test.loc[test['주차가능여부'] == '불가능', '총주차대수'] = test.loc[test['주차가능여부'] == '불가능', '총주차대수'].fillna(0)

train.loc[train['주차가능여부'] == '가능', '총주차대수'] = train.loc[train['주차가능여부'] == '가능', '총주차대수'].fillna(
    train.loc[train['주차가능여부'] == '가능', '총주차대수'].median())
test.loc[test['주차가능여부'] == '가능', '총주차대수'] = test.loc[test['주차가능여부'] == '가능', '총주차대수'].fillna(
    test.loc[test['주차가능여부'] == '가능', '총주차대수'].median())

# 2.6 방수 처리
# 중개사무소별 평균값으로 방수 결측치 대체
train['방수'] = train['방수'].fillna(train.groupby('중개사무소')['방수'].transform('mean'))
test['방수'] = test['방수'].fillna(test.groupby('중개사무소')['방수'].transform('mean'))

# 남아있는 결측치는 전체 평균값으로 대체
train['방수'] = train['방수'].fillna(train['방수'].mean())
test['방수'] = test['방수'].fillna(test['방수'].mean())


# 결측치 확인
print("Train set missing values:")
print(train.isnull().sum())
print("\nTest set missing values:")
print(test.isnull().sum())


Train set missing values:
매물확인방식    0
보증금       0
월세        0
전용면적      0
해당층       0
총층        0
방향        0
방수        0
욕실수       0
주차가능여부    0
총주차대수     0
관리비       0
중개사무소     0
제공플랫폼     0
게재일       0
허위매물여부    0
dtype: int64

Test set missing values:
매물확인방식    0
보증금       0
월세        0
전용면적      0
해당층       0
총층        0
방향        0
방수        0
욕실수       0
주차가능여부    0
총주차대수     0
관리비       0
중개사무소     0
제공플랫폼     0
게재일       0
dtype: int64


In [4]:
# 수정된 train과 test 데이터를 새로운 CSV 파일로 저장
train.to_csv('/apps/study_promptengineerings/dacon/real_estate_fraud/processed_train.csv', index=False)
test.to_csv('/apps/study_promptengineerings/dacon/real_estate_fraud/processed_test.csv', index=False)

print("파일 저장 완료: processed_train.csv와 processed_test.csv")

파일 저장 완료: processed_train.csv와 processed_test.csv
